In [ ]:
import sqlite3
import pprint
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
from os import path

db = sqlite3.connect(path.normpath('D:\\temp\\hf.gnucash'))
cursor = db.cursor()
# start_date = input()
start_date = '2020-01-01'
end_date = '2020-10-01'
query = """
SELECT substr(t.enter_date, 0, 11) AS date,
       a.name AS category,
      (s.value_num / 100.0) AS value
  FROM accounts a,
       splits s,
       transactions t
 WHERE s.account_guid = a.guid AND 
       s.tx_guid = t.guid AND 
       a.account_type = 'EXPENSE' AND 
       strftime('%Y-%m-%d', substr(t.enter_date, 0, 11) ) >= strftime('%Y-%m-%d', '{0}') AND
       strftime('%Y-%m-%d', substr(t.enter_date, 0, 11) ) < strftime('%Y-%m-%d', '{1}')
 ORDER BY date;
""".format(start_date, end_date)
cursor.execute(query)
data = cursor.fetchall()

In [ ]:
df = pd.DataFrame(data, columns=['Date', 'Category', 'Value'])
# df = df.convert_objects(convert_numeric=True)
df = df.astype({'Value':'float64', 'Category':'string'})
df['Date'] = pd.to_datetime(df['Date'], format='%Y-%m-%d')
pt = df.pivot_table('Value', index='Category', columns='Date', fill_value=0, aggfunc=np.sum)
pt = pt.resample('w', axis=1).sum()

In [ ]:

pd.options.display.float_format = '{:.2f}'.format
pt['Average'] = pt.mean(axis=1)
pt.loc['Total']= pt.sum() 
pt

In [ ]:
tt = pt
tt = tt.drop(columns=['Average'])
tt = tt.drop(['Total'])
tt = pt.T
tt.plot(kind='area', logx=True, stacked=True, figsize=(40,30))
